In [2]:
import logging
import time
from pprint import pprint
import ray
import random
from random import randint
import numpy as np

In [3]:
from collections import defaultdict
@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(list)
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)
    
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(60):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))  

In [4]:
if ray.is_initialized:
    ray.shutdown()
context = ray.init(logging_level=logging.ERROR)
pprint(context)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.12', ray_version='2.0.0.dev0', ray_commit='{{RAY_COMMIT_SHA}}', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-04-12_17-36-24_075332_63548/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-04-12_17-36-24_075332_63548/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-04-12_17-36-24_075332_63548', 'metrics_export_port': 55021, 'gcs_address': '127.0.0.1:63184', 'address': '127.0.0.1:63184', 'node_id': '7b1932893136ebdc505d7c89756e53dd861ab5a948a3831d403f163e'})


In [5]:
logging_actor = LoggingActor.remote()
experiment_ids = []
for i in range(3):
    experiment_ids.append(run_experiment.remote(i, logging_actor))

In [7]:
logs = logging_actor.get_logs.remote()
print(ray.get(logs))

{0: ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8', 'On iteration 9', 'On iteration 10', 'On iteration 11', 'On iteration 12', 'On iteration 13', 'On iteration 14', 'On iteration 15', 'On iteration 16', 'On iteration 17', 'On iteration 18', 'On iteration 19', 'On iteration 20', 'On iteration 21', 'On iteration 22', 'On iteration 23', 'On iteration 24', 'On iteration 25', 'On iteration 26', 'On iteration 27', 'On iteration 28', 'On iteration 29', 'On iteration 30', 'On iteration 31', 'On iteration 32', 'On iteration 33'], 2: ['On iteration 0', 'On iteration 1', 'On iteration 2', 'On iteration 3', 'On iteration 4', 'On iteration 5', 'On iteration 6', 'On iteration 7', 'On iteration 8', 'On iteration 9', 'On iteration 10', 'On iteration 11', 'On iteration 12', 'On iteration 13', 'On iteration 14', 'On iteration 15', 'On iteration 16', 'On iteration 17', 'On iteration 18', 'On